# Basic Classification by Income

This notebook would contain basic data cleaning and classification

Source ACS

In [11]:
import pandas as pd

df_msa = pd.read_csv('../data/msa-by-zip.csv')

# Filter out invalid ZIP codes (less than 3 digits, e.g., P.O. Boxes, special-purpose ZIPs)
df_msa = df_msa[df_msa['ZIP CODE'] >= 601]

# Convert ZIP codes to 5-digit strings with leading zeros
df_msa['ZIP CODE'] = df_msa['ZIP CODE'].astype(str).str.zfill(5)

# Standardize column names to lowercase and replace spaces with underscores
df_msa.columns = [col.lower().replace(" ", "_") for col in df_msa.columns]

# Create a subset with only ZIP code and MSA name for merging later
df_subset = df_msa[['zip_code', 'msa_name']]

# Define data cleaning function
def clean_data(df, df_subset):
    """
    Cleans and processes a DataFrame containing ZIP Code Tabulation Area (ZCTA5) and income data, 
    merges it with a subset DataFrame, and returns the cleaned result along with the name of the median income column.

    Parameters:
    -----------
    df : pd.DataFrame
        Raw DataFrame containing income and geographic data. Assumes the first row is a header row, 
        and the first and last columns are to be dropped.
    
    df_subset : pd.DataFrame
        DataFrame containing a 'zip_code' column and an 'msa_name' column to be merged with `df`.
    
    Returns:
    --------
    tuple:
        - pd.DataFrame: Cleaned and merged DataFrame with relevant ZIP code and MSA (Metropolitan Statistical Area) data.
        - str: The column name used for median household income, depending on which format was present in the original data.
    """
    # Use the first row as the header, then reset index and drop the original header row
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)

    # Drop the first and last columns (likely metadata or margin of error)
    df = df.drop([df.columns[0], df.columns[-1]], axis=1)

    # Extract 5-digit ZIP codes from the 'Geographic Area Name' column
    df["zip_code"] = df["Geographic Area Name"].str.extract(r'ZCTA5 (\d{5})')

    # Drop the 'Geographic Area Name' column after extracting the ZIP code
    df = df.drop(columns='Geographic Area Name')

    # Standardize column names: lowercase and underscores instead of spaces
    df.columns = [col.lower().replace(" ", "_") for col in df.columns]

    # Handle one possible format for the median income column
    if 'estimate!!households!!median_income_(dollars)' in df.columns:
        # Convert income to numeric, drop rows with non-numeric values
        df['col_numeric'] = pd.to_numeric(df['estimate!!households!!median_income_(dollars)'], errors='coerce')
        df = df.dropna(subset=['col_numeric']).drop(columns='col_numeric')
        df['estimate!!households!!median_income_(dollars)'] = df['estimate!!households!!median_income_(dollars)'].astype(float)
        median_key = 'estimate!!households!!median_income_(dollars)'

    # Handle alternate format of median income column
    elif 'households!!estimate!!median_income_(dollars)' in df.columns:
        df['col_numeric'] = pd.to_numeric(df['households!!estimate!!median_income_(dollars)'], errors='coerce')
        df = df.dropna(subset=['col_numeric']).drop(columns='col_numeric')
        df['households!!estimate!!median_income_(dollars)'] = df['households!!estimate!!median_income_(dollars)'].astype(float)
        median_key = 'households!!estimate!!median_income_(dollars)'

    # Merge with MSA name data on ZIP code
    df_merged = pd.merge(df, df_subset, on='zip_code', how='left')

    # Reorder columns: zip_code and msa_name first
    col = ['zip_code', 'msa_name']
    df_merged = df_merged[col + [c for c in df_merged.columns if c not in col]]

    # Remove rows that are marked as NONMETROPOLITAN or have no MSA name
    df_merged = df_merged[~df_merged['msa_name'].str.contains('NONMETROPOLITAN', na=False)]
    df_merged = df_merged.dropna(subset=['msa_name'])

    return df_merged, median_key

df_base = pd.read_csv('../data/2011_acs_income_data.csv')

# Clean and process the data using the function
df_base, med_key = clean_data(df_base, df_subset)

# Store the income column name for this specific year
key_2011 = med_key

# Preview the cleaned DataFrame
df_base.head()

# Optional: save cleaned data to CSV
# df.to_csv('2023_acs_income_data_cleaned.csv', index=False)

/var/folders/l5/c83q5w7j3wvcv1kvpg3d5qc80000gn/T/ipykernel_84709/3498039458.py:82: DtypeWarning: Columns (2,3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_base = pd.read_csv('../data/2011_acs_income_data.csv')


,zip_code,msa_name,households!!estimate!!total,households!!margin_of_error!!total,families!!estimate!!total,families!!margin_of_error!!total,married-couple_families!!estimate!!total,married-couple_families!!margin_of_error!!total,nonfamily_households!!estimate!!total,nonfamily_households!!margin_of_error!!total,...,nonfamily_households!!estimate!!percent_imputed!!family_income_in_the_past_12_months,nonfamily_households!!margin_of_error!!percent_imputed!!family_income_in_the_past_12_months,households!!estimate!!percent_imputed!!nonfamily_income_in_the_past_12_months,households!!margin_of_error!!percent_imputed!!nonfamily_income_in_the_past_12_months,families!!estimate!!percent_imputed!!nonfamily_income_in_the_past_12_months,families!!margin_of_error!!percent_imputed!!nonfamily_income_in_the_past_12_months,married-couple_families!!estimate!!percent_imputed!!nonfamily_income_in_the_past_12_months,married-couple_families!!margin_of_error!!percent_imputed!!nonfamily_income_in_the_past_12_months,nonfamily_households!!estimate!!percent_imputed!!nonfamily_income_in_the_past_12_months,nonfamily_households!!margin_of_error!!percent_imputed!!nonfamily_income_in_the_past_12_months
1,00602,"Aguadilla-Isabela-San Sebastian, PR MSA",13500,297,10589,405,7355,381,2911,309,...,(X),(X),(X),(X),(X),(X),(X),(X),8.2,(X)
2,00603,"Aguadilla-Isabela-San Sebastian, PR MSA",17958,439,12961,525,8035,436,4997,470,...,(X),(X),(X),(X),(X),(X),(X),(X),13.4,(X)
4,00610,"Aguadilla-Isabela-San Sebastian, PR MSA",9288,286,7238,342,4670,290,2050,221,...,(X),(X),(X),(X),(X),(X),(X),(X),8.1,(X)
5,00612,"San Juan-Caguas-Guaynabo, PR MSA",23569,528,17926,550,10788,535,5643,407,...,(X),(X),(X),(X),(X),(X),(X),(X),14.0,(X)
6,00616,"San Juan-Caguas-Guaynabo, PR MSA",3551,262,2662,243,1577,200,889,177,...,(X),(X),(X),(X),(X),(X),(X),(X),8.5,(X)


In [12]:
cols = ['households!!estimate!!less_than_$10,000', 'households!!estimate!!$10,000_to_$14,999', 'households!!estimate!!$15,000_to_$24,999']
for c in cols:
    df_base[c] = pd.to_numeric(df_base[c], errors='coerce')
    df_base = df_base.dropna(subset=[c])

df_selected_columns = df_base[cols]

row_sums = df_selected_columns.sum(axis=1)

df_base['per_low_income'] = row_sums

unique_msa = df_base['msa_name'].unique()

output = []

for msa in unique_msa:
    df_specific_msa = df_base[df_base['msa_name'] == msa].set_index('zip_code')

    median_low_income = df_base['per_low_income'].median()

    for zip in list(df_specific_msa.index):
        per_low_income = df_specific_msa.loc[zip, 'per_low_income']

        output.append({'zip_code': zip, 'msa_name': msa, 'per_low_income': per_low_income, 'at_risk_income': per_low_income > median_low_income})


df_output = pd.DataFrame.from_records(output)
df_output.to_csv('../data/low_income_classification_risk.csv', index=False)


In [7]:
import pandas as pd

def per_change(income1, income2):
    """Calculates percent change from income1 to income2 (e.g., 0.1 = 10% increase)."""
    return (income2 / income1) - 1 if income1 != 0 else 0

# Prepare base 2011 data
df_base = df_base.set_index('zip_code')
med_income_2011 = df_base.groupby('msa_name')[key_2011].median().to_dict()

# Placeholder for results
records = []

# Loop from 2012 to 2023
for year in range(2012, 2024):
    print(f"Processing year {year}...")
    
    df_current = pd.read_csv(f'../data/{year}_acs_income_data.csv')
    df_current, df_med_key = clean_data(df_current, df_subset)
    df_current = df_current.set_index('zip_code')
    
    # Get unique MSAs
    msas = df_current['msa_name'].unique()

    for msa in msas:
        df_current_msa = df_current[df_current['msa_name'] == msa]
        median_income_current = df_current_msa[df_med_key].median()
        median_income_base = med_income_2011.get(msa, None)

        # Skip MSA if 2011 data not available
        if median_income_base is None:
            continue

        region_change = per_change(median_income_base, median_income_current)

        # Join current and base data on ZIP for this MSA
        df_base_msa = df_base[df_base['msa_name'] == msa]
        shared_zips = df_current_msa.index.intersection(df_base_msa.index)

        for zip_code in shared_zips:
            income_current = df_current_msa.loc[zip_code, df_med_key]
            income_base = df_base_msa.loc[zip_code, key_2011]

            zip_change = per_change(income_base, income_current)
            gentrified = zip_change > region_change

            records.append({
                'zip_code': zip_code,
                'msa_name': msa,
                'income': income_current,
                'gentrified': gentrified,
                'year': year
            })

# Final DataFrame and export
df_track = pd.DataFrame.from_records(records)
df_track.to_csv('../data/low_income_classification_by_year.csv', index=False)


Processing year 2012...


/var/folders/l5/c83q5w7j3wvcv1kvpg3d5qc80000gn/T/ipykernel_80145/2805901765.py:18: DtypeWarning: Columns (2,3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_current = pd.read_csv(f'../data/{year}_acs_income_data.csv')


Processing year 2013...


/var/folders/l5/c83q5w7j3wvcv1kvpg3d5qc80000gn/T/ipykernel_80145/2805901765.py:18: DtypeWarning: Columns (2,3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_current = pd.read_csv(f'../data/{year}_acs_income_data.csv')


Processing year 2014...


...

/var/folders/l5/c83q5w7j3wvcv1kvpg3d5qc80000gn/T/ipykernel_80145/2805901765.py:18: DtypeWarning: Columns (2,3,34,35,66,67,98,99) have mixed types. Specify dtype option on import or set low_memory=False.
  df_current = pd.read_csv(f'../data/{year}_acs_income_data.csv')


Processing year 2022...


/var/folders/l5/c83q5w7j3wvcv1kvpg3d5qc80000gn/T/ipykernel_80145/2805901765.py:18: DtypeWarning: Columns (2,3,34,35,66,67,98,99) have mixed types. Specify dtype option on import or set low_memory=False.
  df_current = pd.read_csv(f'../data/{year}_acs_income_data.csv')


Processing year 2023...


/var/folders/l5/c83q5w7j3wvcv1kvpg3d5qc80000gn/T/ipykernel_80145/2805901765.py:18: DtypeWarning: Columns (2,3,34,35,66,67,98,99) have mixed types. Specify dtype option on import or set low_memory=False.
  df_current = pd.read_csv(f'../data/{year}_acs_income_data.csv')
